In [1]:
%load_ext autoreload
%autoreload 2

# Extract features, retrain Sherlock and generate predictions.

The script below first downloads the data (roughly 700K samples), then extract features from the raw data values. <br>
If you want to skip this step, you can follow the steps below the feature extraction to load the preprocessed data, 
retrain Sherlock and generate predictions.

In [2]:
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import tensorflow as tf
from ast import literal_eval

from sherlock import helpers
from sherlock.features.preprocessing import extract_features, convert_string_lists_to_lists, prepare_feature_extraction
from sherlock.deploy.train_sherlock import train_sherlock
from sherlock.deploy.predict_sherlock import predict_sherlock

from pympler import muppy, summary
from datetime import datetime

## Download data
This will download the raw values and preprocessed files, the corresponding labels as well as a few other supporting files:
- `download_data()` will download 3.6GB of data into the `data/` directory.
- `prepare_feature_extraction()` will download +/- 800 MB of data into the `features/` directory.

In [3]:
helpers.download_data()
prepare_feature_extraction()

Data was downloaded.
Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.


## Read in raw data
You can skip this step if you want to use a preprocessed data file.

## Report memory usage (can be slow)

In [4]:
report_memory = False

## Extract features
It is important that the string-representations of lists are first converted into lists of strings.
The labels should be a list of semantic types.

In [5]:
import time 

timestr = time.strftime("%Y%m%d-%H%M%S")

X_test_filename_csv = f'../data/work/test_{timestr}.csv'
X_train_filename_csv = f'../data/work/train_{timestr}.csv'
X_validation_filename_csv = f'../data/work/validation_{timestr}.csv'

In [6]:
import io
import csv
from datetime import datetime
from functional import pseq
from sherlock.functional import as_py_str, to_literal, randomise_sample, extract_features, normalise_string_whitespace, keys_on_first
from pyarrow.parquet import ParquetFile


def keys_to_csv(keys):
    with io.StringIO() as output:
        writer = csv.writer(output, quoting=csv.QUOTE_NONNUMERIC)
        writer.writerow(keys)

        return output.getvalue()
    

def load_parquet_values(path):
    pf = ParquetFile(source = path)
    row_df = pf.read_row_group(0)
    
    return row_df['values']


def extract_features_to_csv(output_path, parquet_values):
    # incompatible with keys_on_first
    verify_keys = False
    first_keys = None
    i = 0

    start = datetime.now()

    print(f'Starting {output_path} at {start}')

    with open(output_path, "w") as outfile:
        # Comparable performance with using pool.imap directly, but the code is *much* cleaner
        for keys, values_str in pseq(parquet_values, processes=6, partition_size=10)\
            .map(as_py_str)\
            .map(to_literal)\
            .map(randomise_sample)\
            .map(normalise_string_whitespace)\
            .map(extract_features)\
            .map(keys_on_first): # to-do: make this function a partial, and pass in the verify_keys
                i = i+1

                if first_keys is None:
                    first_keys = keys
                    first_keys_str = keys_to_csv(keys)

                    print(f'Exporting {len(first_keys)} column features')
                    
                    outfile.write(keys_to_csv(keys))
                    outfile.write('\n')
                elif verify_keys: # incompatible with keys_on_first
                    keys_str = ','.join(keys)
                    if first_keys_str != keys_str:
                        key_list = list(keys)

                        print(f'keys are NOT equal. k1 len={len(first_keys)}, k2 len={len(keys)}')

                        for idx, k1 in enumerate(first_keys):
                            k2 = key_list[idx]

                            if k1 != k2:
                                print(f'{k1} != {k2}')

                outfile.write(values_str)
                outfile.write('\n')

    print(f'Finished. Processed {i} rows in {datetime.now() - start}')

### PREPARATION

In [7]:
# ensure embedding initialisation is outside of timing for extract_features
from sherlock.features.word_embeddings import initialise_word_embeddings
from sherlock.features.paragraph_vectors import initialise_pretrained_model, initialise_nltk
from sherlock.features.preprocessing import prepare_feature_extraction

prepare_feature_extraction()
initialise_word_embeddings()
initialise_pretrained_model(400)
initialise_nltk()

Preparing feature extraction by downloading 2 files:
        
 ../sherlock/features/glove.6B.50d.txt and 
 ../sherlock/features/par_vec_trained_400.pkl.docvecs.vectors_docs.npy.
        
All files for extracting word and paragraph embeddings are present.
Initialising word embeddings
Initialise Word Embeddings process took 0:00:05.729550 seconds.
Initialise Doc2Vec Model, 400 dim, process took 0:00:02.248020 seconds. (filename = ../sherlock/features/par_vec_retrained_400.pkl)
Initialised NLTK, process took 0:00:00.132775 seconds.


[nltk_data] Downloading package punkt to /Users/lowecg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lowecg/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## EXTRACT FEATURES TO CSV (NEW METHOD)

### TEST SET

In [8]:
values = load_parquet_values("../data/raw/test_values.parquet")

extract_features_to_csv(X_test_filename_csv, values)

values = None

Starting ../data/work/test_20210102-095844.csv at 2021-01-02 09:58:53.257343
Exporting 1578 column features
Finished. Processed 137353 rows in 0:06:50.696652


In [9]:
# Finished. Processed 137353 rows in 0:14:53.196073

In [10]:
print(f'Finished at {datetime.now()}')

Finished at 2021-01-02 10:05:44.170768


### TRAIN SET

In [11]:
values = load_parquet_values("../data/raw/train_values.parquet")

extract_features_to_csv(X_train_filename_csv, values)

values = None

Starting ../data/work/train_20210102-095844.csv at 2021-01-02 10:07:34.760579
Exporting 1578 column features
Finished. Processed 412059 rows in 0:19:31.704850


In [12]:
print(f'Finished at {datetime.now()}')

Finished at 2021-01-02 10:27:06.631137


### VALIDATION SET

In [13]:
values = load_parquet_values("../data/raw/val_values.parquet")

extract_features_to_csv(X_validation_filename_csv, values)

values = None

Starting ../data/work/validation_20210102-095844.csv at 2021-01-02 10:27:07.542500
Exporting 1578 column features
Finished. Processed 137353 rows in 0:06:28.035298


In [14]:
print(f'Finished at {datetime.now()}')

Finished at 2021-01-02 10:33:35.696038


## EXTRACT FEATURES TO CSV (**OLD** METHOD) 

### TEST SET (OLD METHOD)

In [8]:
if report_memory:
    all_objects = muppy.get_objects()
    sum1 = summary.summarize(all_objects)
    # Prints out a summary of the large objects
    summary.print_(sum1)

In [9]:
test_samples = pd.read_parquet('../data/raw/test_values.parquet')
test_labels = pd.read_parquet('../data/raw/test_labels.parquet')

In [ ]:

test_samples_converted, y_test = convert_string_lists_to_lists(test_samples, test_labels, "values", "type")

In [12]:
# free memory
test_samples = None
test_labels = None

In [13]:
test_samples_converted.head()

20368     [Central Missouri, unattached, unattached, Kan...
664102    [95, 100, 95, 89, 84, 91, 88, 94, 75, 78, 90, ...
366813    [Katie Crews, Christian Hiraldo, Alex Estrada,...
530567    [Christian, Non-Christian, Unreported, Jewish,...
176253    [AAF-McQuay Canada Inc., AAF-McQuay Canada Inc...
Name: values, dtype: object

In [14]:
# output "head" 
y_test[:5]

['affiliation', 'weight', 'jockey', 'religion', 'company']

### Given that feature extraction can take long, we only take the first 100 samples.

In [15]:
y_test_subset = y_test[:100]

In [16]:
if report_memory:
    all_objects = muppy.get_objects()
    sum1 = summary.summarize(all_objects)
    # Prints out a summary of the large objects
    summary.print_(sum1)

In [17]:
%load_ext line_profiler

In [25]:
from sherlock.features.preprocessing import extract_features

start = datetime.now()

#extract_features(X_test_filename, test_samples_converted.head(n=100))

%lprun -m sherlock.features.preprocessing X_test=extract_features('deleteme.csv', test_samples_converted.head(n=100)) 

print(f'Extract Features (test) process took {datetime.now() - start} seconds.')

Extracting Features:   6%|▌         | 6/100 [00:00<00:01, 54.82it/s]

Exporting 1578 column features


Extracting Features: 100%|██████████| 100/100 [00:01<00:00, 71.93it/s]


Extract Features (test) process took 0:00:01.437484 seconds.


In [26]:
test_samples_converted = None

In [22]:
from sherlock.features.helpers import generate_chars_col
generate_chars_col()

In [ ]:
import gc

gc.collect()

In [ ]:
if report_memory:
    all_objects = muppy.get_objects()
    sum1 = summary.summarize(all_objects)
    # Prints out a summary of the large objects
    summary.print_(sum1)

In [ ]:
# over all, without memory management
# Extract Features (test) process took 3:40:25.799880 seconds.

In [ ]:
# Baseline
# Extract Features (test) process took 0:11:04.137081 seconds.

# Iterations
# Extract Features (test) process took 0:00:56.671353 seconds. (cache word embeddings)
# Extract Features (test) process took 0:00:13.523261 seconds. (cache Doc2Vec)
# Extract Features (test) process took 0:00:03.674007 seconds. (loads of tweaks, use np.array for stats)
# Extract Features (test) process took 0:00:03.262298 seconds. (manually compute counts)
# Extract Features (test) process took 0:00:02.853031 seconds. (replace series.str.count in BoW, use series.tolist() in paragraph vectors )
# Extract Features (test) process took 0:00:01.718025 seconds. (compute mean, variance, skew and kurtosis together)
# Extract Features (test) process took 0:00:01.437484 seconds. (compute sum, min and max together, nunique replaced with len(set(series))), use statistics.median not np.median)

### TRAIN SET (OLD METHOD)

In [193]:
train_samples = pd.read_parquet('../data/raw/train_values.parquet')
train_labels = pd.read_parquet('../data/raw/train_labels.parquet')

In [ ]:
train_samples_converted, y_train = convert_string_lists_to_lists(train_samples, train_labels, "values", "type")

In [ ]:
# free memory
train_samples = None
train_labels = None

In [ ]:
y_train_subset = y_train[:100]

In [ ]:
start = datetime.now()

extract_features(X_train_filename, train_samples_converted)

print(f'Extract Features (train) process took {datetime.now() - start} seconds.')

In [ ]:
train_samples_converted = None

### VALIDATION SET (OLD METHOD)

In [194]:
validation_samples = pd.read_parquet('../data/raw/val_values.parquet')
validation_labels = pd.read_parquet('../data/raw/val_labels.parquet')

In [ ]:
validation_samples_converted, y_validation = convert_string_lists_to_lists(validation_samples, validation_labels, "values", "type")

In [ ]:
# free memory
validation_samples = None
validation_labels = None

In [ ]:
y_validation_subset = y_validation[:100]

In [ ]:
start = datetime.now()

extract_features(X_validation_filename, validation_samples_converted)

print(f'Extract Features (validation) process took {datetime.now() - start} seconds.')

In [ ]:
validation_samples_converted = None

### Read Locally Processed Features

In [ ]:
timestr = '20201224-105345'

X_test_filename_csv = f'../data/work/test_{timestr}.csv'
X_train_filename_csv = f'../data/work/train_{timestr}.csv'
X_validation_filename_csv = f'../data/work/validation_{timestr}.csv'

In [15]:
start = datetime.now()

X_test = pd.read_csv(X_test_filename_csv, dtype=np.float32)

print(f'Load Features (test) process took {datetime.now() - start} seconds.')

Load Features (test) process took 0:00:30.423832 seconds.


In [16]:
X_test.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.00000,...,-0.019620,-0.005751,-0.091799,-0.005163,0.026949,0.126209,0.044317,0.046805,0.283753,-0.172713
1,1.0,0.0,0.368421,0.337950,0.0,2.0,0.0,7.0,0.742677,1.32687,...,-0.059000,0.119227,0.053944,-0.001340,0.131212,0.041129,0.051907,-0.097376,0.382744,-0.083757
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.00000,...,-0.099030,-0.017373,0.095235,-0.115544,0.025403,0.036343,-0.058500,0.138506,0.282834,-0.015050
3,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.00000,...,-0.134826,-0.011558,0.105958,0.034837,0.075483,-0.059032,0.027906,0.027777,0.097653,-0.065297
4,1.0,0.0,0.020080,0.035741,0.0,2.0,0.0,5.0,96.521599,9.78415,...,-0.334457,-0.091482,-0.374800,-0.848663,-0.562704,0.551798,0.344166,0.099641,0.830072,0.208357


In [17]:
start = datetime.now()

X_train = pd.read_csv(X_train_filename_csv, dtype=np.float32)

print(f'Load Features (train) process took {datetime.now() - start} seconds.')

Load Features (train) process took 0:01:37.263390 seconds.


In [18]:
X_train.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.009202,-0.063930,0.010675,0.008881,-0.045870,-0.003592,0.081800,-0.030984,0.135367,-0.094720
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.098076,0.079344,-0.067548,0.128541,0.108601,-0.032248,0.086094,-0.006857,0.347734,-0.073678
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.024619,0.215026,-0.165963,0.036715,0.493099,0.221508,0.409486,-0.434891,0.204209,-0.338041
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.038246,-0.047774,0.149694,0.056400,-0.141803,-0.050657,0.107286,-0.078970,0.430949,-0.154992
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.0,0.0,...,-0.227529,0.101622,0.135879,-0.165079,0.192703,0.089738,0.030012,-0.044039,0.427989,-0.063405


In [19]:
start = datetime.now()

X_validation = pd.read_csv(X_validation_filename_csv, dtype=np.float32)

print(f'Load Features (validation) process took {datetime.now() - start} seconds.')

Load Features (validation) process took 0:00:30.623068 seconds.


In [20]:
X_validation.head()

,n_[0]-agg-any,n_[0]-agg-all,n_[0]-agg-mean,n_[0]-agg-var,n_[0]-agg-min,n_[0]-agg-max,n_[0]-agg-median,n_[0]-agg-sum,n_[0]-agg-kurtosis,n_[0]-agg-skewness,...,par_vec_390,par_vec_391,par_vec_392,par_vec_393,par_vec_394,par_vec_395,par_vec_396,par_vec_397,par_vec_398,par_vec_399
0,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.00000,...,0.308804,0.218672,0.298720,-0.054650,-0.038117,0.394701,0.328886,-0.213662,0.359657,-0.045453
1,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.00000,...,-0.115212,0.337192,0.064066,0.014705,-0.001189,0.089293,0.316962,-0.254808,0.420812,-0.266880
2,1.0,0.0,0.25000,0.187500,0.0,1.0,0.0,10.0,-0.666667,1.15470,...,0.168952,0.081204,-0.370821,-0.069921,0.030297,0.080933,0.005761,0.032868,0.053071,0.038055
3,0.0,0.0,0.00000,0.000000,0.0,0.0,0.0,0.0,-3.000000,0.00000,...,-0.040249,0.106651,-0.039631,-0.061941,0.071239,0.109763,0.136131,0.034865,0.246903,0.048431
4,1.0,0.0,0.09434,0.311855,0.0,4.0,0.0,5.0,42.275002,6.53059,...,-0.060758,0.714009,0.116055,0.089381,-0.016312,-0.120640,0.604058,-0.263433,0.848908,-0.157347


## Impute NaN values with feature means

In [21]:
start = datetime.now()

train_columns_means = pd.DataFrame(X_train.mean()).transpose()

print(f'Transpose process took {datetime.now() - start} seconds.')

Transpose process took 0:00:21.022117 seconds.


In [22]:
start = datetime.now()

X_train.fillna(train_columns_means.iloc[0], inplace=True)
X_validation.fillna(train_columns_means.iloc[0], inplace=True)
X_test.fillna(train_columns_means.iloc[0], inplace=True)

train_columns_means=None

print(f'FillNA process took {datetime.now() - start} seconds.')

FillNA process took 0:00:01.856752 seconds.


In [23]:
start = datetime.now()

X_train.to_parquet('train.parquet', engine='pyarrow', compression='snappy')
X_validation.to_parquet('validation.parquet', engine='pyarrow', compression='snappy')
X_test.to_parquet('test.parquet', engine='pyarrow', compression='snappy')

print(f'Save parquet process took {datetime.now() - start} seconds.')

Save parquet process took 0:00:57.697665 seconds.


In [24]:
X_test = None

In [25]:
import gc

gc.collect()

40

In [26]:
print(f'Finished at {datetime.now()}')

Finished at 2020-12-30 17:52:40.703314


## Restore Pickled Session ready for training

In [ ]:
X_train = pd.read_csv('train.csv')
X_validation = pd.read_csv('validation.csv')

In [ ]:
y_train = pd.read_parquet('../data/raw/train_labels.parquet').values.flatten()
y_validation = pd.read_parquet('../data/raw/val_labels.parquet').values.flatten()

In [ ]:
y_train

## Retrain sherlock
The model can be retrained using the code below. The model is currently restricted to be trained on 78 classes, the code of the model architecture will soon be added for adjusting this.

In [ ]:
print(f'Started at {datetime.now()}')

train_sherlock(X_train, y_train, X_validation, y_validation, nn_id='retrained_sherlock3');

print('Trained and saved new model.')
print(f'Finished at {datetime.now()}')


## Generate predictions with a model
If you want to use the pretrained Sherlock model `nn_id` set to "sherlock".

If you want to use another model, you can use the identifier corresponding to that model.

**Note**: There is a bug somewhere in the refactored code which affects the model predictions, this should be fixed soon.

In [ ]:
X_test = pd.read_csv('test.csv')

In [ ]:
y_test = pd.read_parquet('../data/raw/test_labels.parquet').values.flatten()

In [ ]:
predicted_labels = predict_sherlock(X_test, nn_id='retrained_sherlock3')

In [ ]:
print(f'prediction count {len(predicted_labels)}, type = {type(predicted_labels)}')

In [ ]:
size=len(y_test)

# Should be fully deterministic too.
f1_score(y_test[:size], predicted_labels[:size], average="weighted")

In [ ]:
pd.Series(predicted_labels).nunique()

In [ ]:
pd.Series(y_test).nunique()

In [ ]:
predicted_labels[:25]

In [ ]:
y_test[:25]

In [ ]:
set(y_test)

In [ ]:
from collections import Counter

size = len(y_test)
mismatches = list()

for idx, k1 in enumerate(y_test[:size]):
    k2 = predicted_labels[idx]

    if k1 != k2:
        mismatches.append(k1)
#        if k1 in ('name'):
#            print(f'[{idx}] expected "{k1}" but predicted "{k2}"')
        
f1 = f1_score(y_test[:size], predicted_labels[:size], average="weighted")
print(f'Total mismatches: {len(mismatches)} (F1 score: {f1})')

data = Counter(mismatches)
data.most_common()   # Returns all unique items and their counts

In [ ]:
test_samples = pd.read_parquet('../data/raw/test_values.parquet')

In [ ]:
idx = 123758
converted = test_samples.iloc[idx].apply(literal_eval).to_list()

print(f'Predicted "{predicted_labels[idx]}", actual label "{y_test[idx]}". Actual values:\n{converted}')

## Generate predictions with preprocessed data using Sherlock

Requires the data to be downloaded from Google Drive (see first step in notebook).

In [ ]:
X_test_preprocessed = pd.read_parquet("../data/data/processed/X_test.parquet")
y_test_preprocessed = pd.read_parquet("../data/data/processed/y_test.parquet").reset_index(drop=True)

In [ ]:
X_test_preprocessed.head()

In [ ]:
#X_test_preprocessed.to_csv('test_preprocessed.csv')

In [ ]:
X_test.head()

In [ ]:
y_test_preprocessed.head()

In [ ]:
predicted_labels = predict_sherlock(X_test_preprocessed.head(n=25), 'sherlock')

In [ ]:
f1_score(y_test_preprocessed.head(n=25), predicted_labels, average='weighted')

In [ ]:
predicted_labels[:10]

In [ ]:
y_test_preprocessed.head(n=10)

In [ ]:
first_keys = X_test_preprocessed.columns
first_keys_str = ','.join(first_keys)

keys = ','.join(X_test.columns)
if first_keys_str == keys:
    print('Keys are equal')
else:
    key_list = list(X_test.columns)

    print(f'keys are NOT equal. k1 len={len(first_keys)}, k2 len={len(key_list)}')

    for idx, k1 in enumerate(first_keys):
        k2 = key_list[idx]

        if k1 == k2:
            print(f'{k1} == {k2}')
        else:
            print(f'{k1} != {k2}')